In [1]:
import pyspark
import os
from graphframes import *
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
%sh wget https://datasets.imdbws.com/title.basics.tsv.gz
wget https://datasets.imdbws.com/title.principals.tsv.gz
wget https://datasets.imdbws.com/title.ratings.tsv.gz
wget https://datasets.imdbws.com/name.basics.tsv.gz
wget https://datasets.imdbws.com/title.episode.tsv.gz

--2019-05-10 17:37:55-- https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.116, 13.224.2.40, 13.224.2.93, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104685428 (100M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

 0K .......... .......... .......... .......... .......... 0% 4.73M 21s
 50K .......... .......... .......... .......... .......... 0% 8.97M 16s
 100K .......... .......... .......... .......... .......... 0% 13.5M 13s
 150K .......... .......... .......... .......... .......... 0% 15.9M 11s
 200K .......... .......... .......... .......... .......... 0% 19.6M 10s
 250K .......... .......... .......... .......... .......... 0% 27.2M 9s
 300K .......... .......... .......... .......... .......... 0% 30.4M 8s
 350K .......... .......... .......... .......... .......... 0% 24.7M 8s
 400K .......... .......... .......... .......... .......... 0% 33.6M 7s
 450K .......... .......... .......... .......... .......... 0% 41.8M 7s
 500K .......... .......... .......... .......... .......... 0% 43.3M 6s
 550K .......... .......... .......... .......... .......... 0% 38.1M 6s
 600K .......... .......... .......... .......... .......... 0% 58.5M 6s
 650K .......... .......... .......... .......... .......... 0% 62.6M 5s
 700K .......... .......... .......... .......... .......... 0% 65.5M 5s
 750K .......... .......... .......... .......... .......... 0% 48.8M 5s
 800K .......... .......... .......... .......... .......... 0% 67.2M 5s
 850K .......... .......... .......... .......... .......... 0% 77.3M 5s
 900K .......... .......... .......... .......... .......... 0% 81.9M 4s
 950K .......... .......... .......... .......... .......... 0% 58.3M 4s
 1000K .......... .......... .......... .......... .......... 1% 166M 4s
 1050K .......... .......... .......... .......... .......... 1% 68.4M 4s
 1100K .......... .......... .......... .......... .......... 1% 92.9M 4s
 1150K .......... .......... .......... .......... .......... 1% 79.3M 4s
 1200K .......... .......... .......... .......... .......... 1% 105M 4s
 1250K .......... .......... .......... .......... .......... 1% 140M 3s
 1300K .......... .......... .......... .......... .......... 1% 103M 3s
 1350K .......... .......... .......... .......... .......... 1% 90.2M 3s
 1400K .......... .......... .......... .......... .......... 1% 107M 3s
 1450K .......... .......... .......... .......... .......... 1% 192M 3s
 1500K .......... .......... .......... .......... .......... 1% 117M 3s
 1550K .......... .......... .......... .......... .......... 1% 116M 3s
 1600K .......... .......... .......... .......... .......... 1% 124M 3s
 1650K .......... .......... .......... .......... .......... 1% 122M 3s
 1700K .......... .......... .......... .......... .......... 1% 163M 3s
 1750K .......... .......... .......... .......... .......... 1% 140M 3s
 1800K .......... .......... .......... .......... .......... 1% 136M 3s
 1850K .......... .......... .......... .......... .......... 1% 139M 3s
 1900K .......... .......... .......... .......... .......... 1% 115M 3s
 1950K .......... .......... .......... .......... .......... 1% 101M 3s
 2000K .......... .......... .......... .......... .......... 2% 158M 2s
 2050K .......... .......... .......... .......... .......... 2% 119M 2s
 2100K .......... .......... .......... .......... .......... 2% 123M 2s
 2150K .......... .......... .......... .......... .......... 2% 116M 2s
 2200K .......... .......... .......... .......... .......... 2% 120M 2s
 2250K .......... .......... .......... .......... .......... 2% 123M 2s
 2300K .......... .......... .......... .......... .......... 2% 127M 2s
 2350K .......... .......... .......... .......... .......... 2% 94.0M 2s
 2400K .......... .......... .......... .......... .......... 2% 128M 2s
 2450K .......... .......... 

In [3]:
%sh ls -l

total 583420
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 731 May 10 17:37 derby.log
drwxr-xr-x 3 root root 4096 May 10 17:37 eventlogs
drwxr-xr-x 2 root root 4096 May 10 17:37 logs
-rw-r--r-- 1 root root 182175918 May 9 13:24 name.basics.tsv.gz
-rw-r--r-- 1 root root 104685428 May 9 13:24 title.basics.tsv.gz
-rw-r--r-- 1 root root 22083935 May 9 13:24 title.episode.tsv.gz
-rw-r--r-- 1 root root 283853181 May 9 13:24 title.principals.tsv.gz
-rw-r--r-- 1 root root 4598151 May 9 13:24 title.ratings.tsv.gz

In [4]:
%sh gunzip *.gz

In [5]:
%sh rm -rf *.gz
ls -l

total 2600612
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 731 May 10 17:37 derby.log
drwxr-xr-x 3 root root 4096 May 10 17:37 eventlogs
drwxr-xr-x 2 root root 4096 May 10 17:37 logs
-rw-r--r-- 1 root root 559480645 May 9 13:24 name.basics.tsv
-rw-r--r-- 1 root root 495861353 May 9 13:24 title.basics.tsv
-rw-r--r-- 1 root root 102135339 May 9 13:24 title.episode.tsv
-rw-r--r-- 1 root root 1489694307 May 9 13:24 title.principals.tsv
-rw-r--r-- 1 root root 15834349 May 9 13:24 title.ratings.tsv

In [6]:
%sh pwd

/databricks/driver

In [7]:
# 1 a) is here

In [8]:
df_name_basics = spark.read.option('sep','\t').option('header','true').option('inferSchema','true').csv('file:/databricks/driver/name.basics.tsv')

# df_name_basics.cache()
print(df_name_basics.count())
df_name_basics.head()

9309072
 Out[ 9 ]: Row(nconst='nm0000001', primaryName='Fred Astaire', birthYear='1899', deathYear='1987', primaryProfession='soundtrack,actor,miscellaneous', knownForTitles='tt0050419,tt0072308,tt0053137,tt0043044')

In [9]:
df_title_basics = spark.read.option('sep','\t').option('header','true').option('inferSchema','true').csv('file:/databricks/driver/title.basics.tsv')

# df_title_basics.cache()
print(df_title_basics.count())
df_title_basics.head()

5844238
 Out[ 10 ]: Row(tconst='tt0000001', titleType='short', primaryTitle='Carmencita', originalTitle='Carmencita', isAdult=0, startYear='1894', endYear='\\N', runtimeMinutes='1', genres='Documentary,Short')

In [10]:
# df_episode_basics = spark.read.option('sep','\t').option('header','true').option('inferSchema','true').csv('file:/databricks/driver/title.episode.tsv')

# df_episode_basics.cache()
# print(df_episode_basics.count())
# df_episode_basics.head()

In [11]:
df_title_principals = spark.read.option('sep','\t').option('header','true').option('inferSchema','true').csv('file:/databricks/driver/title.principals.tsv')

# df_title_principals = df_title_principals[df_title_principals['category'] == 'actor']
# df_title_principals.cache()
print(df_title_principals.count())
df_title_principals.head()

33546789
 Out[ 12 ]: Row(tconst='tt0000001', ordering=1, nconst='nm1588970', category='self', job='\\N', characters='["Herself"]')

In [12]:
df_title_ratings = spark.read.option('sep','\t').option('header','true').option('inferSchema','true').csv('file:/databricks/driver/title.ratings.tsv')

# df_title_ratings.cache()
print(df_title_ratings.count())
df_title_ratings.head()

931165
 Out[ 13 ]: Row(tconst='tt0000001', averageRating=5.8, numVotes=1497)

In [13]:
# 1 b)
_1_b_i_title_ratings = df_title_basics.join(df_title_ratings, df_title_basics.tconst==df_title_ratings.tconst, 'inner')

_1_b_i_grouped_ratings = _1_b_i_title_ratings.groupby('genres').sum('numVotes')

maxVotes = _1_b_i_grouped_ratings.groupby().max('sum(numVotes)').collect()[0].asDict()['max(sum(numVotes))']

mostVotedGenres = _1_b_i_grouped_ratings[_1_b_i_grouped_ratings['sum(numVotes)'] == maxVotes].collect()#[0].asDict()['genres']

display(mostVotedGenres)

# print('most voted genres is %s' % mostVotedGenres)
# TODO - sort genre combos

genres,sum(numVotes)
"Action,Adventure,Sci-Fi",38236542


In [14]:
# df_title_basics

In [15]:
_1_b_ii_title_ratings_500 = _1_b_i_title_ratings[_1_b_i_title_ratings.numVotes >= 500]

_1_b_ii_grouped_ratings = _1_b_ii_title_ratings_500.groupby('genres').avg('averageRating')

maxAvgRating = _1_b_ii_grouped_ratings.groupby().max('avg(averageRating)').collect()[0].asDict()['max(avg(averageRating))']

maxAvgRatingGenres = _1_b_ii_grouped_ratings[_1_b_ii_grouped_ratings['avg(averageRating)'] == maxAvgRating].collect()

display(maxAvgRatingGenres)

# print('highest average rating genre combo is %s' % maxAvgRatingGenres)

genres,avg(averageRating)
"Music,News,Short",9.1
"Biography,History,Mystery",9.1


In [16]:
# 1 b iii
from pyspark.sql.functions import split
from pyspark.sql.functions import col

# df_split_genresRDD = 
genre_counts_rdd = df_title_basics.filter(df_title_basics['genres'] != '\\N').withColumn('temp', split(col('genres'), ',')).rdd.flatMap(lambda row: [(g,1) for g in row[9]]).reduceByKey(lambda a, b: a+b)

maxCount = genre_counts_rdd.reduce(lambda a, b: a if a[1] > b[1] else b)

print('Most common genre is %s at %d' % (maxCount))

# df_split_genresRDD.take(1)

Most common genre is Drama at 1548382

In [17]:
# 1 b iv
genre_year_rdd = df_title_basics.filter(df_title_basics['genres'] != '\\N').filter(df_title_basics['startYear'] != '\\N').filter(df_title_basics['startYear'] >= 2000).withColumn('temp', split(col('genres'), ',')).rdd.flatMap(lambda row: [(row[5], {g: 1}) for g in row[9]])

from collections import Counter

# Thank you stack overflow
genre_year_rdd.reduceByKey(lambda a, b: dict(sum((Counter(x) for x in [a, b]), Counter()))).map(lambda y: (y[0], max(y[1], key=y[1].get), y[1][max(y[1], key=y[1].get)])).collect()

Out[ 18 ]: 
[('2017', 'Drama', 73986),
 ('2015', 'Comedy', 70808),
 ('2025', 'Action', 2),
 ('2014', 'Comedy', 69185),
 ('2012', 'Drama', 63530),
 ('2008', 'Drama', 48956),
 ('2115', 'Drama', 1),
 ('2018', 'Drama', 65707),
 ('2013', 'Comedy', 66358),
 ('2003', 'Drama', 30587),
 ('2020', 'Drama', 393),
 ('2019', 'Drama', 22229),
 ('2007', 'Drama', 46722),
 ('2022', 'Action', 8),
 ('2023', 'Action', 1),
 ('2001', 'Drama', 25138),
 ('2005', 'Drama', 36197),
 ('2002', 'Drama', 26119),
 ('2004', 'Drama', 30547),
 ('2021', 'Animation', 24),
 ('2016', 'Comedy', 73728),
 ('2010', 'Drama', 55342),
 ('2011', 'Drama', 59558),
 ('2006', 'Drama', 41214),
 ('2009', 'Drama', 50680),
 ('2024', 'Fantasy', 2),
 ('2000', 'Drama', 22204)]

In [18]:
# 2)
principals_r = df_title_principals.withColumnRenamed('tconst', 'tc')
_2_a_i_df = df_title_basics.join(principals_r, df_title_basics.tconst == principals_r.tc, 'inner')
_2_a_i_df.head()

Out[ 19 ]: Row(tconst='tt0000658', titleType='short', primaryTitle="The Puppet's Nightmare", originalTitle='Le cauchemar de Fantoche', isAdult=0, startYear='1908', endYear='\\N', runtimeMinutes='2', genres='Animation,Short', tc='tt0000658', ordering=1, nconst='nm0169871', category='director', job='\\N', characters='\\N')

In [19]:
_2_a_ii_df = _2_a_i_df[(_2_a_i_df.category=='actor') | (_2_a_i_df.category=='actress')]
_2_a_ii_df = _2_a_ii_df[_2_a_ii_df.isAdult==0]
_2_a_ii_df = _2_a_ii_df[_2_a_ii_df.titleType=='movie']

valid_actors = df_name_basics[(df_name_basics.birthYear >= 1920) & (df_name_basics.birthYear != '\\N') & (df_name_basics.deathYear == '\\N')].select('nconst')

_2_a_ii_df = _2_a_ii_df.join(valid_actors, 'nconst', 'inner')#.select(_2_a_ii_df.columns)
# not technically just removing ¯\_(ツ)_/¯
display(_2_a_ii_df)

nconst,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tc,ordering,category,job,characters
nm0000198,tt0110116,movie,Immortal Beloved,Immortal Beloved,0,1994,\N,121,"Biography,Drama,Music",tt0110116,1,actor,\N,"[""Ludwig van Beethoven""]"
nm0000198,tt1345836,movie,The Dark Knight Rises,The Dark Knight Rises,0,2012,\N,164,"Action,Thriller",tt1345836,4,actor,\N,"[""Commissioner Gordon""]"
nm0000198,tt0097125,movie,Criminal Law,Criminal Law,0,1988,\N,117,"Crime,Thriller",tt0097125,1,actor,\N,"[""Ben Chase""]"
nm0000198,tt3239932,movie,True Crime: The Movie,True Crime: The Movie,0,2012,\N,87,Documentary,tt3239932,3,actor,\N,"[""Rocky"",""Agent Masterson""]"
nm0000198,tt0208874,movie,The Contender,The Contender,0,2000,\N,126,"Drama,Thriller",tt0208874,2,actor,\N,"[""Shelly Runyon""]"
nm0000198,tt0100519,movie,Rosencrantz & Guildenstern Are Dead,Rosencrantz & Guildenstern Are Dead,0,1990,\N,117,"Comedy,Drama",tt0100519,1,actor,\N,"[""Rosencrantz""]"
nm0000198,tt6111574,movie,The Woman in the Window,The Woman in the Window,0,2019,\N,\N,"Crime,Drama,Mystery",tt6111574,3,actor,\N,"[""Alistair Russell""]"
nm0000198,tt0118571,movie,Air Force One,Air Force One,0,1997,\N,124,"Action,Drama,Thriller",tt0118571,2,actor,\N,"[""Ivan Korshunov""]"
nm0000198,tt2103281,movie,Dawn of the Planet of the Apes,Dawn of the Planet of the Apes,0,2014,\N,130,"Action,Adventure,Drama",tt2103281,1,actor,\N,"[""Dreyfus""]"
nm0000198,tt2461520,movie,Man Down,Man Down,0,2015,\N,90,"Drama,Thriller",tt2461520,3,actor,\N,"[""Counselor Peyton""]"


In [20]:
# 2 a iii
n_actors = len(_2_a_ii_df.groupby('nconst').count().collect())

print('there are %d actors and actresses' % n_actors)

there are 82567 actors and actresses

In [21]:
# _2_a_ii_df.select('tconst', 'nconst').withColumnRenamed('tconst', 'tc1').withColumnRenamed('nconst', 'nc1')

In [22]:
# 2 a iv
df1 = _2_a_ii_df.select('tconst', 'nconst').withColumnRenamed('tconst', 'tc1').withColumnRenamed('nconst', 'nc1')
df2 = _2_a_ii_df.select('tconst', 'nconst').withColumnRenamed('tconst', 'tc2').withColumnRenamed('nconst', 'nc2')


title_actors_df = df1.join(df2, df1.tc1 == df2.tc2, 'inner').select('tc1', 'nc1', 'nc2')
title_actors_df = title_actors_df[title_actors_df.nc1 != title_actors_df.nc2]
# title_actors_df.head(5)

_2_a_iv_df = title_actors_df.select('nc1', 'nc2').drop_duplicates()
_2_a_iv_df.cache()
display(_2_a_iv_df)

nc1,nc2
nm0051880,nm0502262
nm0343857,nm0092573
nm0230931,nm0153641
nm0001696,nm0507659
nm0009629,nm0796396
nm0516272,nm0009629
nm0148661,nm0811660
nm0001166,nm0329640
nm0560364,nm0004760
nm0000800,nm0004760


In [23]:
# 2 b
# _2_a_iv_df is the edges
# vertices can be derived
import graphframes.graphframe as gfm

vertices = _2_a_iv_df.select('nc1').withColumnRenamed('nc1', 'id').drop_duplicates()
edges = _2_a_iv_df.withColumnRenamed('nc1', 'src').withColumnRenamed('nc2', 'dst')
vertices = vertices.join(df_name_basics, vertices.id==df_name_basics.nconst, 'inner')
vertices.cache()
edges.cache()
edges.count()

Out[ 24 ]: 697252

In [24]:
gf = gfm.GraphFrame(vertices, edges)
pageRank = gf.pageRank(resetProbability=0.15, maxIter=5)

In [25]:
asc = pageRank.vertices.sort('pagerank')
dsc = pageRank.vertices.sort('pagerank', ascending=False)

dsc.show(10)

+---------+---------+----------------+---------+---------+--------------------+--------------------+------------------+
 id| nconst| primaryName|birthYear|deathYear| primaryProfession| knownForTitles| pagerank|
+---------+---------+----------------+---------+---------+--------------------+--------------------+------------------+
nm0000616|nm0000616| Eric Roberts| 1956| \N|actor,producer,so...|tt5836706,tt04135...|36.759142130289895|
nm0000514|nm0000514| Michael Madsen| 1957| \N|actor,producer,wr...|tt0114508,tt01052...|21.775346614138165|
nm0001744|nm0001744| Tom Sizemore| 1961| \N|actor,producer,wr...|tt0114558,tt02650...| 20.78182230615126|
nm0001803|nm0001803| Danny Trejo| 1944| \N|actor,producer,so...|tt0113277,tt09856...|19.045442612892323|
nm0001698|nm0001698| John Savage| 1949| \N|actor,producer,so...|tt0077416,tt00792...| 16.9465152229148|
nm0000448|nm0000448| Lance Henriksen| 1940| \N| actor,miscellaneous|tt0088247,tt00906...|15.970209493461997|
nm0000367|nm0000367|Gérard Depardieu| 1948| \N|actor,producer,so...|tt0103594,tt01207...|15.631625250892366|
nm0004193|nm0004193| Debbie Rochon| 1968| \N|actress,producer,...|tt1401631,tt22747...|15.550953787220886|
nm0626259|nm0626259| Franco Nero| 1941| \N|actor,producer,di...|tt0099423,tt18537...|15.516411903498538|
nm0261724|nm0261724| Joe Estevez| 1946| \N|actor,producer,di...|tt5823428,tt01028...|15.347116827941544|
+---------+---------+----------------+---------+---------+--------------------+--------------------+------------------+
only showing top 10 rows

In [26]:
asc[['id', 'pagerank']].show(10)

+---------+-------------------+
 id| pagerank|
+---------+-------------------+
nm0762643|0.17402296660365238|
nm7987433|0.17541362501791205|
nm0080295|0.17613196476160653|
nm0012868|0.17622465353640757|
nm0008711|0.17622465353640757|
nm0451809|0.17634043943215838|
nm3576158|0.17634043943215838|
nm6000269|0.17756494572690174|
nm0481568|0.17756494572690174|
nm2056531|0.17814071849265173|
+---------+-------------------+
only showing top 10 rows

In [27]:
pageRank.vertices

Out[ 28 ]: DataFrame[id: string, nconst: string, primaryName: string, birthYear: string, deathYear: string, primaryProfession: string, knownForTitles: string, pagerank: double]

In [28]:
# 2 b ii
v = pageRank.vertices[['id', 'pagerank']]
rank_names = v.join(df_name_basics, v.id == df_name_basics.nconst, 'inner')
rank_names.groupby('birthYear').avg('pagerank').collect()

Out[ 29 ]: 
[Row(birthYear='1953', avg(pagerank)=1.1780221115824703),
 Row(birthYear='1957', avg(pagerank)=1.161810916415134),
 Row(birthYear='1987', avg(pagerank)=0.8063334827733523),
 Row(birthYear='1956', avg(pagerank)=1.1439325674791045),
 Row(birthYear='1936', avg(pagerank)=1.1225951695574847),
 Row(birthYear='2012', avg(pagerank)=0.2931623076971451),
 Row(birthYear='1958', avg(pagerank)=1.1308307048059651),
 Row(birthYear='1943', avg(pagerank)=1.1174426539921656),
 Row(birthYear='1972', avg(pagerank)=1.0240275414687983),
 Row(birthYear='1931', avg(pagerank)=1.0787045737360386),
 Row(birthYear='1926', avg(pagerank)=0.9115614008308042),
 Row(birthYear='1988', avg(pagerank)=0.7705077362073375),
 Row(birthYear='1938', avg(pagerank)=1.1353540894968386),
 Row(birthYear='1932', avg(pagerank)=0.9215129588816258),
 Row(birthYear='1977', avg(pagerank)=0.8893819997990826),
 Row(birthYear='1971', avg(pagerank)=1.0017100225328197),
 Row(birthYear='2014', avg(pagerank)=0.5265692873054528),
 Row(birthYear='1984', avg(pagerank)=0.8492145340402681),
 Row(birthYear='1982', avg(pagerank)=0.8802874250648168),
 Row(birthYear='1941', avg(pagerank)=1.2026556452234776),
 Row(birthYear='2013', avg(pagerank)=1.3193281168255648),
 Row(birthYear='2005', avg(pagerank)=0.5534640313742817),
 Row(birthYear='2000', avg(pagerank)=0.5825264438900003),
 Row(birthYear='1965', avg(pagerank)=1.1124814603746684),
 Row(birthYear='1962', avg(pagerank)=1.079389053002845),
 Row(birthYear='1954', avg(pagerank)=1.1740034152650929),
 Row(birthYear='1930', avg(pagerank)=1.1393973494472536),
 Row(birthYear='1981', avg(pagerank)=0.9068725410710202),
 Row(birthYear='1940', avg(pagerank)=1.153533859898834),
 Row(birthYear='1978', avg(pagerank)=0.9291441507505374),
 Row(birthYear='1974', avg(pagerank)=0.9561070343726928),
 Row(birthYear='1959', avg(pagerank)=1.1249793998793727),
 Row(birthYear='2002', avg(pagerank)=0.6386047935665092),
 Row(birthYear='2009', avg(pagerank)=0.4236016853017306),
 Row(birthYear='1995', avg(pagerank)=0.6641908560720288),
 Row(birthYear='1964', avg(pagerank)=1.1089609148126751),
 Row(birthYear='1946', avg(pagerank)=1.212036939864286),
 Row(birthYear='2006', avg(pagerank)=0.5679772546953045),
 Row(birthYear='1927', avg(pagerank)=0.9074228370611813),
 Row(birthYear='1921', avg(pagerank)=0.6606010882821729),
 Row(birthYear='1976', avg(pagerank)=0.9498892362785565),
 Row(birthYear='1942', avg(pagerank)=1.0862120677291753),
 Row(birthYear='1935', avg(pagerank)=1.1303212048680003),
 Row(birthYear='1947', avg(pagerank)=1.150587058805848),
 Row(birthYear='1968', avg(pagerank)=1.050074586632224),
 Row(birthYear='1967', avg(pagerank)=1.0512829799611936),
 Row(birthYear='1922', avg(pagerank)=0.6704416366132013),
 Row(birthYear='1924', avg(pagerank)=0.8626585838903806),
 Row(birthYear='2004', avg(pagerank)=0.5706032695795141),
 Row(birthYear='1939', avg(pagerank)=1.1839964657076867),
 Row(birthYear='1989', avg(pagerank)=0.7790056772972862),
 Row(birthYear='2011', avg(pagerank)=0.6181210745925116),
 Row(birthYear='1992', avg(pagerank)=0.7407521958235341),
 Row(birthYear='1961', avg(pagerank)=1.1124262730871912),
 Row(birthYear='1951', avg(pagerank)=1.1450205961359132),
 Row(birthYear='1966', avg(pagerank)=1.0868454366224884),
 Row(birthYear='2008', avg(pagerank)=0.44070780940131166),
 Row(birthYear='1955', avg(pagerank)=1.2418785119278135),
 Row(birthYear='1999', avg(pagerank)=0.5921881270309832),
 Row(birthYear='1963', avg(pagerank)=1.1252152061886633),
 Row(birthYear='1994', avg(pagerank)=0.6827300495961908),
 Row(birthYear='1950', avg(pagerank)=1.1484185995919465),
 Row(birthYear='1997', avg(pagerank)=0.6684948744259983),
 Row(birthYear='1973', avg(pagerank)=0.9837114928084525),
 Row(birthYear='1925', avg(pagerank)=0.9175730361849675),
 Row(birthYear='2007', avg(pagerank)=0.47971580411530435),
 Row(birthYear='1996', avg(pagerank)=0.7018222700995698),
 Row(birthYear='1983', avg(pagerank)=0.8622000055409841),
 Row(birthYear='1969', avg(pagerank)=1.027464787

In [29]:
# 2 c
bot_10 = spark.createDataFrame(asc.head(10)).withColumnRenamed('id', '_id')
top_10 = spark.createDataFrame(dsc.head(10)).withColumnRenamed('id', '_id')

In [30]:
gf.degrees.join(top_10, top_10._id==gf.degrees.id)[['id', 'degree']].show()

+---------+------+
 id|degree|
+---------+------+
nm0000616| 852|
nm0001698| 440|
nm0004193| 238|
nm0261724| 270|
nm0000514| 550|
nm0000367| 528|
nm0626259| 400|
nm0001803| 456|
nm0000448| 392|
nm0001744| 500|
+---------+------+

In [31]:
gf.degrees.join(bot_10, bot_10._id==gf.degrees.id)[['id', 'degree']].show()

+---------+------+
 id|degree|
+---------+------+
nm0012868| 2|
nm3576158| 2|
nm0762643| 2|
nm6000269| 2|
nm7987433| 2|
nm0008711| 2|
nm0451809| 2|
nm0080295| 2|
nm2056531| 2|
nm0481568| 2|
+---------+------+

In [32]:
# 3 a i
# df_name_basics[(df_name_basics.primaryName=='Kevin Bacon') & (df_name_basics.birthYear != '\\N')].collect()

gf.vertices.filter('primaryName == "Kevin Bacon"')[['nconst', 'primaryName']].show()
#nm0000102

+---------+-----------+
 nconst|primaryName|
+---------+-----------+
nm0000102|Kevin Bacon|
+---------+-----------+

In [33]:
paths = gf.shortestPaths(landmarks=['nm0000102'])
paths

Out[ 34 ]: DataFrame[id: string, nconst: string, primaryName: string, birthYear: string, deathYear: string, primaryProfession: string, knownForTitles: string, distances: map<string,int>]

In [34]:
# 3 a iii
from pyspark.sql.functions import explode
summary = paths.select('id', 'primaryName', explode('distances')).groupby('value').count()
summary.show()

+-----+-----+
value|count|
+-----+-----+
 1| 126|
 6| 2097|
 3|17817|
 5|12652|
 9| 18|
 4|28565|
 8| 45|
 7| 300|
 10| 4|
 11| 2|
 2| 3258|
 0| 1|
+-----+-----+

In [35]:
# unconnected to kevin bacon
paths.count() - summary.groupby().sum('count').collect()[0][0]

Out[ 103 ]: 6204

In [36]:
# 4 a ii
movies_df = _2_a_ii_df[['tconst', 'nconst', 'primaryTitle']].dropDuplicates()
cast_ranks = pageRank.vertices[['id', 'pagerank']]

_4_a_ii = movies_df.join(cast_ranks, cast_ranks.id == movies_df.nconst, 'inner').groupby(['tconst', 'primaryTitle']).avg('pagerank')
display(_4_a_ii)

tconst,primaryTitle,avg(pagerank)
tt0424859,The Ninth Cloud,7.697940431720883
tt0169035,Los mochileros,1.799433444974127
tt0212815,Big Eden,2.7213693808678134
tt0128855,Zbor periculos,1.4306203928199095
tt5159688,Otrazheniye,1.8639808720543922
tt0045796,Husband and Wife,0.6928858365463373
tt2266819,The Yellow Rose,2.3522384271764283
tt0842926,The Kids Are All Right,4.603618407704029
tt0410626,Slingshot,3.617122641922136
tt1716740,Down by the Water,6.9351727416678335


In [37]:
# 4 a iii
# gf.degrees
_4_a_iii = movies_df.join(gf.degrees, gf.degrees.id==movies_df.nconst, 'inner').groupby(['tconst', 'primaryTitle']).avg('degree')
display(_4_a_iii)

tconst,primaryTitle,avg(degree)
tt0424859,The Ninth Cloud,201.5
tt0169035,Los mochileros,34.0
tt0212815,Big Eden,69.0
tt0128855,Zbor periculos,26.666666666666668
tt5159688,Otrazheniye,37.6
tt0045796,Husband and Wife,18.0
tt2266819,The Yellow Rose,53.0
tt0842926,The Kids Are All Right,171.5
tt0410626,Slingshot,109.5
tt1716740,Down by the Water,204.5


In [38]:
# 4 a iv
# Dataframe with: title_id, title Name, avg cast pagerank, avg cast degree, rating
_4_a_iv_df = df_title_ratings[['tconst', 'averageRating']].withColumnRenamed('tconst', 'title_id').withColumnRenamed('averageRating', 'rating')
_4_a_iv_df = _4_a_iv_df.join(_4_a_ii, _4_a_iv_df.title_id==_4_a_ii.tconst, 'inner')[['title_id', 'avg(pagerank)', 'rating']]

_4_a_iv_df = _4_a_iii.join(_4_a_iv_df, _4_a_iv_df.title_id==_4_a_iii.tconst, 'inner')[['title_id', 'primaryTitle', 'avg(pagerank)', 'avg(degree)', 'rating']]
_4_a_iv_df

Out[ 173 ]: DataFrame[title_id: string, primaryTitle: string, avg(pagerank): double, avg(degree): double, rating: double]

In [39]:
from pyspark.ml.linalg import Vectors
def transData(data):
  # Combine columns to a dense vector (excluding the last column)
  dataFeaturesRDD = data.rdd.map(lambda r: [Vectors.dense(r[2:4]),r[-1]])
  
  # Convert the RDD back to a DataFrame, labelling the columns
  featuresDF =  dataFeaturesRDD.toDF(['features','label'])
  
  return featuresDF

In [40]:
_4_a_iv_df

Out[ 175 ]: DataFrame[title_id: string, primaryTitle: string, avg(pagerank): double, avg(degree): double, rating: double]

In [41]:
dataLR = transData(_4_a_iv_df)
display(dataLR)

features,label
"List(1, 2, List(), List(0.2171130888289111, 4.0))",7.4
"List(1, 2, List(), List(0.4833262374096371, 2.0))",5.7
"List(1, 2, List(), List(2.211174763101435, 8.0))",5.8
"List(1, 2, List(), List(0.7131222455909777, 12.0))",7.3
"List(1, 2, List(), List(1.5548933783813974, 29.0))",7.2
"List(1, 2, List(), List(0.6398529072250448, 6.0))",6.2
"List(1, 2, List(), List(0.7864283047139129, 9.0))",6.7
"List(1, 2, List(), List(0.2523600873591062, 2.0))",5.8
"List(1, 2, List(), List(1.167515815473209, 42.0))",6.4
"List(1, 2, List(), List(3.2502912970921933, 76.0))",7.4


In [42]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()

# 4 a iv model
model = lr.fit(dataLR)

# there's no v) item ¯\_(ツ)_/¯

# 4 a vi
print("Model intercept: %r, coefficient: %r" % (model.intercept, model.coefficients[0]))

>>>> Model intercept: 6.0353589330473945, coefficient: -0.082826843805465164